**Single Cell Analysis**


In this tutorial, we will be analyzing the a dataset of Peripheral Blood Mononuclear Cells (PBMC) freely available from 10X Genomics. There are 2,700 single cells that were sequenced on the Illumina NextSeq 500. 

The example raw data can be found here (https://cf.10xgenomics.com/samples/cell/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz)



First, install and Load Required packages

In [ ]:
install.packages('Seurat')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bitops’, ‘gtools’, ‘caTools’, ‘BH’, ‘sitmo’, ‘globals’, ‘listenv’, ‘parallelly’, ‘plyr’, ‘zoo’, ‘htmlwidgets’, ‘lazyeval’, ‘crosstalk’, ‘promises’, ‘RcppTOML’, ‘here’, ‘gplots’, ‘reshape2’, ‘gridExtra’, ‘RcppArmadillo’, ‘httpuv’, ‘xtable’, ‘fontawesome’, ‘sourcetools’, ‘later’, ‘spatstat.data’, ‘spatstat.random’, ‘spatstat.utils’, ‘spatstat.sparse’, ‘abind’, ‘tensor’, ‘goftest’, ‘deldir’, ‘polyclip’, ‘FNN’, ‘RSpectra’, ‘dqrng’, ‘cowplot’, ‘fitdistrplus’, ‘future’, ‘future.apply’, ‘ggrepel’, ‘ggridges’, ‘ica’, ‘igraph’, ‘irlba’, ‘leiden’, ‘lmtest’, ‘matrixStats’, ‘miniUI’, ‘patchwork’, ‘pbapply’, ‘plotly’, ‘png’, ‘RANN’, ‘RcppAnnoy’, ‘reticulate’, ‘ROCR’, ‘Rtsne’, ‘scattermore’, ‘sctransform’, ‘SeuratObject’, ‘shiny’, ‘spatstat.core’, ‘spatstat.geom’, ‘uwot’, ‘RcppEigen’, ‘RcppProgress’




In [ ]:
install.packages('SeuratData')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“package ‘SeuratData’ is not available for this version of R

A version of this package for your version of R might be available elsewhere,
see the ideas at
https://cran.r-project.org/doc/manuals/r-patched/R-admin.html#Installing-packages”


In [ ]:
install.packages('ggplot2')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
install.packages('patchwork')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
install.packages('dplyr')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
library(Seurat)
#library(SeuratData)
library(ggplot2)
library(patchwork)
library(dplyr)

Attaching SeuratObject


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




First, Read10X() function reads in the output of the cellranger pipeline from 10X, returning a unique molecular identified (UMI) count matrix. The values in this matrix represent the number of molecules for each feature (i.e. gene; row) that are detected in each cell (column).

In [ ]:
# Load the PBMC dataset
pbmc.data <- Read10X(data.dir = "/")
# Initialize the Seurat object with the raw (non-normalized data).
pbmc <- CreateSeuratObject(counts = pbmc.data, project = "pbmc3k", min.cells = 3, min.features = 200)
pbmc

ERROR: ignored

View matrix data

In [ ]:
# Lets view a few genes in the first 10 cells
pbmc.data[c("CD3D", "TCL1A", "MS4A1"), 1:30]

view metadata

In [ ]:
# Show QC metrics for the first 5 cells
head(pbmc@meta.data, 5)

**QC and selection of good quality cells**

**A few QC metrics commonly used:**
1. The number of unique genes detected in each cell.

- Low-quality cells or empty droplets will often have very few genes
Cell doublets or multiplets may exhibit an aberrantly high gene count
Similarly, the total number of molecules detected within a cell (correlates strongly with unique genes)

2. The percentage of reads that map to the mitochondrial genome
- Low-quality / dying cells often exhibit extensive mitochondrial contamination

We calculate mitochondrial QC metrics with the PercentageFeatureSet() function, which calculates the percentage of counts originating from a set of features
We use the set of all genes starting with MT- as a set of mitochondrial genes

In [ ]:
# The [[ operator can add columns to object metadata. This is a great place to stash QC stats
pbmc[["percent.mt"]] <- PercentageFeatureSet(pbmc, pattern = "^MT-")

In [ ]:
# Show QC metrics for the first 5 cells
head(pbmc@meta.data, 5)

In [ ]:
# Visualize QC metrics as a violin plot
VlnPlot(pbmc, features = c("nFeature_RNA", "nCount_RNA", "percent.mt"), ncol = 3)

In [ ]:
# FeatureScatter is typically used to visualize feature-feature relationships, but can be used
# for anything calculated by the object, i.e. columns in object metadata, PC scores etc.

plot1 <- FeatureScatter(pbmc, feature1 = "nCount_RNA", feature2 = "percent.mt")
plot2 <- FeatureScatter(pbmc, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")
plot1 + plot2

**Qality control filters**

We filter cells that have unique feature counts over 2,500 or less than 200
We filter cells that have >5% mitochondrial counts

Remove empty/broken cells (with high mitrochondrial content) and doublets/multiplets (by removing high RNA features >2000/2500)

In [ ]:
pbmc <- subset(pbmc, subset = nFeature_RNA > 200 & nFeature_RNA < 2500 & percent.mt < 5)

**Normalizing the data**

After removing unwanted cells from the dataset, the next step is to normalize the data. 

By default, we employ a global-scaling normalization method “LogNormalize” that normalizes the feature expression measurements for each cell by the total expression, multiplies this by a scale factor (10,000 by default), and log-transforms the result.

In [ ]:
#Normalize data
pbmc_norm <- NormalizeData(pbmc, normalization.method = "LogNormalize", scale.factor = 10000)
#or 
#pbmc_norm <- NormalizeData(pbmc)

**Identify Highly variable features (feature selection)** 

Now, we will identify a subset of features that exhibit high cell-to-cell variation in the dataset (i.e, they are highly expressed in some cells, and lowly expressed in others). These genes in downstream analysis helps to highlight biological signal in single-cell datasets.

For this purpose, we can use FindVariableFeatures() function. By default, we return 2,000 features per dataset. These will be used in downstream analysis, like PCA.

In [ ]:
pbmc_hv <- FindVariableFeatures(pbmc_norm, selection.method = "vst", nfeatures = 2000)

# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(pbmc_hv), 10)

Visualize top features

In [ ]:
# plot variable features with and without labels
plot1 <- VariableFeaturePlot(pbmc_hv)
plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)
plot1 + plot2

**Scaling the data**

Now, we will apply ‘scaling’ that is a standard pre-processing step prior to dimensional reduction techniques like PCA using ScaleData() function:

It will Shifts the expression of each gene, so that the mean expression across cells is 0 Scales the expression of each gene, so that the variance across cells is 1.

This step gives equal weight in downstream analyses, so that highly-expressed genes do not dominate

In [ ]:
#scale data
all.genes <- rownames(pbmc_hv)
pbmc_scaled <- ScaleData(pbmc_hv, features = all.genes)

**Linear dimensional Reduction**

Next we will perform PCA on the scaled data. By default, only the previously determined variable features are used as input

In [ ]:
#perform PCA
pbmc_pca <- RunPCA(pbmc_scaled, features = VariableFeatures(object = pbmc_scaled))

In [ ]:
# Examine and visualize PCA results a few different ways
print(pbmc_pca[["pca"]], dims = 1:5, nfeatures = 5)

In [ ]:
VizDimLoadings(pbmc_pca, dims = 1:2, reduction = "pca")

In [ ]:
#PCA scatterplot 
DimPlot(pbmc_pca, reduction = "pca")

**DimHeatmap**

DimHeatmap() allows for easy exploration of the primary sources of heterogeneity in a dataset, and can be useful when trying to decide which PCs to include for further downstream analyses. 

Both cells and features are ordered according to their PCA scores. 

In [ ]:
DimHeatmap(pbmc_pca, dims = 1, cells = 500, balanced = TRUE)

In [ ]:
DimHeatmap(pbmc_pca, dims = 1:15, cells = 500, balanced = TRUE)

**Determine the ‘dimensionality’ of the dataset**

To overcome the extensive technical noise in any single feature for scRNA-seq data, Seurat clusters cells based on their PCA scores, with each PC essentially representing a ‘metafeature’ that combines information across a correlated feature set. 

The top principal components therefore represent a robust compression of the dataset. However, how many components should we choose to include? 10? 20? 100?

In Macosko et al, we implemented a resampling test inspired by the JackStraw procedure. We randomly permute a subset of the data (1% by default) and rerun PCA, constructing a ‘null distribution’ of feature scores, and repeat this procedure. We identify ‘significant’ PCs as those who have a strong enrichment of low p-value features.

**NOTE:** This process can take a long time for big datasets, comment out for expediency. More

approximate techniques such as those implemented in ElbowPlot() can be used to reduce


In [ ]:
# computation time
pbmc_p <- JackStraw(pbmc_pca, num.replicate = 100)


In [ ]:
pbmc_score <- ScoreJackStraw(pbmc_p, dims = 1:20)

**JackStrawPlot**

The JackStrawPlot() function provides a visualization tool for comparing the distribution of p-values for each PC with a uniform distribution (dashed line). 

‘Significant’ PCs will show a strong enrichment of features with low p-values (solid curve above the dashed line).

In [ ]:
JackStrawPlot(pbmc_score, dims = 1:20)

 In this case it appears that there is a sharp drop-off in significance after the first 10-12 PCs.

**Elbow plot**

An alternative heuristic method generates an ‘Elbow plot’: a ranking of principle components based on the percentage of variance explained by each one (ElbowPlot() function). 

In [ ]:
ElbowPlot(pbmc_score)

In this example, we can observe an ‘elbow’ around PC9-10, suggesting that the majority of true signal is captured in the first 10 PCs.

**Clustering of cells**

Seurat v3 applies a graph-based clustering approach, building upon initial strategies in (Macosko et al). Importantly, the distance metric which drives the clustering analysis (based on previously identified PCs) remains the same. However, in this approach of clustering embed cells in a graph structure - for example a K-nearest neighbor (KNN) graph, with edges drawn between cells with similar feature expression patterns, and then attempt to partition this graph into highly interconnected ‘quasi-cliques’ or ‘communities’.

As in PhenoGraph, here, a KNN graph based on the euclidean distance in PCA space, and refine the edge weights between any two cells based on the shared overlap in their local neighborhoods (Jaccard similarity). This step is performed using the FindNeighbors() function, and takes as input the previously defined dimensionality of the dataset (first 10 PCs).

To cluster the cells,  next, we will apply modularity optimization techniques such as the Louvain algorithm (default) or SLM [SLM, Blondel et al., Journal of Statistical Mechanics], to iteratively group cells together, with the goal of optimizing the standard modularity function. 

The FindClusters() function implements this procedure, and contains a resolution parameter that sets the ‘granularity’ of the downstream clustering, with increased values leading to a greater number of clusters. We find that setting this parameter between 0.4-1.2 typically returns good results for single-cell datasets of around 3K cells. Optimal resolution often increases for larger datasets. The clusters can be found using the Idents() function.

In [ ]:
pbmc_nn <- FindNeighbors(pbmc_score, dims = 1:10) ## taken first 10 pca components


In [ ]:
#find clusters
pbmc_cluster <- FindClusters(pbmc_nn, resolution = 0.5)

In [ ]:
# Look at cluster IDs of the first 5 cells
head(Idents(pbmc), 5)

**Perform non-linear dimensional reduction (UMAP/tSNE)**
Seurat allows several non-linear dimensional reduction techniques, such as tSNE and UMAP, to visualize and explore these datasets.

 The goal of these algorithms is to learn the underlying manifold of the data in order to place similar cells together in low-dimensional space. Cells within the graph-based clusters determined above should co-localize on these dimension reduction plots

**UMAP**

In [ ]:
# If you haven't installed UMAP, you can do so via reticulate::py_install(packages =
# 'umap-learn')
pbmc_umap <- RunUMAP(pbmc_cluster, dims = 1:10)

In [ ]:
# note that you can set `label = TRUE` or use the LabelClusters function to help label
# individual clusters
DimPlot(pbmc_umap, reduction = "umap")

Now, save the object at this point so that it can easily be loaded back in without having to rerun the computationally intensive steps performed above

In [ ]:
#save data or object
saveRDS(pbmc_umap, file = "pbmc_umap.rds")

**Finding differentially expressed features (cluster biomarkers)**

Seurat help you find markers that define clusters via differential expression. 

By default, it identifies positive and negative markers of a single cluster (specified in ident.1), compared to all other cells. 

`FindAllMarkers()` automates this process for all clusters, but you can also test groups of clusters vs. each other, or against all cells.

The `min.pct` argument requires a feature to be detected at a minimum percentage in either of the two groups of cells, and the `thresh.test` argument requires a feature to be differentially expressed (on average) by some amount between the two groups. You can set both of these to 0, but with a dramatic increase in time - since this will test a large number of features that are unlikely to be highly discriminatory. 

As another option to speed up these computations, `max.cells.per.ident` can be set. This will downsample each identity class to have no more cells than whatever this is set to. While there is generally going to be a loss in power, the speed increases can be significant and the most highly differentially expressed features will likely still rise to the top.

**find all markers of cluster 2**


In [ ]:
cluster2.markers <- FindMarkers(pbmc_umap, ident.1 = 2, min.pct = 0.25)
head(cluster2.markers, n = 5)

In [ ]:
# find all markers distinguishing cluster 5 from clusters 0 and 3
cluster5.markers <- FindMarkers(pbmc_umap, ident.1 = 5, ident.2 = c(0, 3), min.pct = 0.25)
head(cluster5.markers, n = 5)

**Find markers for every cluster compared to all remaining cells**

In [ ]:
# find markers for every cluster compared to all remaining cells, report only the positive ones
pbmc.markers <- FindAllMarkers(pbmc_umap, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)
pbmc.markers %>%
    group_by(cluster) %>%
    slice_max(n = 2, order_by = avg_log2FC)

Seurat has several tests for differential expression which can be set with the test.use parameter (see our DE vignette for details). For example, the ROC test returns the ‘classification power’ for any individual marker

In [ ]:
cluster0.markers <- FindMarkers(pbmc_umap, ident.1 = 2, logfc.threshold = 0.25, test.use = "roc", only.pos = TRUE)

In [ ]:
head(cluster0.markers,5)

***Visualizing marker expression ***

`VlnPlot()` (shows expression probability distributions across clusters), and `FeaturePlot()` (visualizes feature expression on a PCA plot) are our most commonly used visualizations. 

In [ ]:
VlnPlot(pbmc_umap, features = c("MS4A1", "CD79A"))

In [ ]:
# you can plot raw counts as well
VlnPlot(pbmc_umap, features = c("MS4A1", "CD79A"), slot = "counts", log = TRUE)

In [ ]:
#feature plot
FeaturePlot(pbmc_umap, features = c("MS4A1", "GNLY", "CD3E", "CD14", "FCER1A", "FCGR3A", "LYZ", "PPBP",
    "CD8A"))

**Heatmap**

`DoHeatmap() `generates an expression heatmap for given cells and features. In this case, we are plotting the top 20 markers (or all markers if less than 20) for each cluster.

In [ ]:
#heatmap for top markers
pbmc.markers %>%
    group_by(cluster) %>%
    top_n(n = 10, wt = avg_log2FC) -> top10
DoHeatmap(pbmc_umap, features = top10$gene) + NoLegend()

**Assigning cell type identity to clusters**

we can use canonical markers to easily match the unbiased clustering to known cell types

In [ ]:
new.cluster.ids <- c("Naive CD4 T", "CD14+ Mono", "Memory CD4 T", "B", "CD8 T", "FCGR3A+ Mono",
    "NK", "DC", "Platelet")
names(new.cluster.ids) <- levels(pbmc_umap)

In [ ]:
#rename clusters with cell types
pbmc_umap_new <- RenameIdents(pbmc_umap, new.cluster.ids)

#plot dimplot with cell type
DimPlot(pbmc_umap_new, reduction = "umap", label = TRUE, pt.size = 0.5) + NoLegend()


In [ ]:
#save object based on cell type
saveRDS(pbmc_umap_new, file = "pbmc3k_final_umap_cells.rds")